# Klasifikasi Berita Detik

Setelah melakukan crawling data pada detik, dan memberi label kategori. Selanjutnya adalah tahap membentuk model untuk keperluan klasifikasi

Import library pandas

In [1]:
import csv
import pandas as pd

Import data hasil crawling

In [2]:
termFreq = pd.read_csv("crawling_alldetik_uas_full.csv")
termFreq.head()


,Judul,Isi,Kategori
0,"Kevin Sanjaya Izin Absen dari Pelatnas, Jaga I...",Pebulutangkis ganda putra Kevin Sanjaya Sukamu...,sport
1,Menpora Optimistis Indonesia Loloskan Banyak A...,Menpora Dito Ariotedjo optimistis Indonesia me...,sport
2,Ranking Terbaru Race to Paris 2024: Dejan/Glo ...,Federasi Bulutangkis Dunia (BWF) kembali meril...,sport
3,"Lomba Lari di Berbagai Negara, 4 Atlet Pakai O...","Kembali gunakan tajuk 'Your Flying Machine', O...",sport
4,"Jadi Kontestan Baru IBL, Kesatria Bengawan Sol...",Kompetisi Indonesia Basketball League (IBL) mu...,sport


Mengecek apakah ada data yang bernilai null

In [3]:
termFreq.isnull().sum()

Judul       0
Isi         1
Kategori    3
dtype: int64

Agar dataset lebih rapi maka data yang memiliki nilai null di-drop

In [4]:
termFreq = termFreq.dropna()

## Preprocessing
Dalam tahap ini, text akan dilakukan cleaning, agar menghasilkan data yang lebih bersih dan lebih mudah diekstraksi informasi.
Kali ini, cleaning data yang akan dilakukan adalah penghapusan kurung, penghapusan url, penghapusan karakter khusus, dan penghapusan stopwords

Import library yang dibutuhkan

In [5]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer
import re, string, unicodedata
from string import punctuation

In [6]:
# Menghapus kurung
def hapus_kurung(text):
  return re.sub('\[[^]]*\]', '', text)

# Menghapus url
def delete_url(text):
  return re.sub(r'http/S+', '', text)
# Menghapus simbol khusus
def remove_specialchars(text):
  pattern=r'[^a-zA-z0-9\s]'
  text = re.sub(pattern,'', text)
  return text
# Menghapus stopwords dengan nltk serta mengubah text menjadi lowercase
nltk.download('stopwords')
stop = set(stopwords.words('indonesian'))
punctuation = list(string.punctuation)
stop.update(punctuation)

def remove_stopwords(text):
  res = []
  for i in text.split():
    if i.strip().lower() not in stop and i.strip().lower().isalpha():
      res.append(i.strip().lower())
  return " ".join(res)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Menyatukan fungsi cleaning yang telah dibuat tadi menjadi satu fungsi preprocessing

In [7]:
def preprocessing_text(text):
  text = hapus_kurung(text)
  text = delete_url(text)
  text = remove_specialchars(text)
  text = remove_stopwords(text)
  return text


Menjalankan fungsi ke dataset. Dengan menggunakan fungsi .apply  dari pandas. Dalam hal ini pada kolom Isi

In [8]:
termFreq['Isi'] = termFreq['Isi'].apply(preprocessing_text)

Menampilkan data yang sudah di preprocessing

In [9]:
termFreq

,Judul,Isi,Kategori
0,"Kevin Sanjaya Izin Absen dari Pelatnas, Jaga I...",pebulutangkis ganda putra kevin sanjaya sukamu...,sport
1,Menpora Optimistis Indonesia Loloskan Banyak A...,menpora dito ariotedjo optimistis indonesia me...,sport
2,Ranking Terbaru Race to Paris 2024: Dejan/Glo ...,federasi bulutangkis dunia bwf merilis ranking...,sport
3,"Lomba Lari di Berbagai Negara, 4 Atlet Pakai O...",tajuk your flying machine ortuseight luncurkan...,sport
4,"Jadi Kontestan Baru IBL, Kesatria Bengawan Sol...",kompetisi indonesia basketball league ibl musi...,sport
...,...,...,...
1086,"Demi Makan Gratis Setahun, Pria Ini Rela Habis...",pria menantang makan hidangan restoran tantang...,food
1087,6 Minuman Enak dan Segar Ini Bisa Menurunkan T...,asupan makanan jenis minuman menurunkan tekana...,food
1088,Gokil! Tanboy Kun Masak dan Mukbang 1 Kg Wagyu...,tanboy kun mencicipi makanan menarik kali yout...,food
1089,"Muncul Tren Aneh, Nasi hingga Mie Instan Dicam...",tren makanan silih berganti tren makanan aneh ...,food


Menyimpan data dalam csv

In [10]:
termFreq.to_csv('preprocessing_alldetik_uas_full.csv', index=False)

Import data yang sudah preprocessing

## Modelling

In [11]:
df = pd.read_csv('preprocessing_alldetik_uas_full.csv')

Import tf-idf vectorizer dari library scikit-learn

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

Dikarenakan yang akan digunakan hanya kolom 'Isi' dan 'Kategori', maka hanya diambil dua kolom tersebut

In [13]:
x = df['Isi'] # Kolom fitur
y = df['Kategori']  # Kolom target


Inisialisasi vectorizer

In [14]:
tfidfvectorizer = TfidfVectorizer()

Fit data ke tfidf

In [15]:
tfidf_fit = tfidfvectorizer.fit(x)
tfidf_text = tfidf_fit.transform(x)

Menampilkan tf-idf

In [16]:
tfidf_tokens = tfidfvectorizer.get_feature_names_out()
df_tfidfvect = pd.DataFrame(data = tfidf_text.toarray(),columns = tfidf_tokens)
df_tfidfvect

,aa,aaaaa,aakarshi,aakasep,aan,aangeliaa,aangeliaadi,aarhus,aarhusdalam,aaron,...,zulfi,zulfikri,zulkieflimansyah,zullies,zulliesmeski,zumba,zunyou,zuppa,zurich,zus
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1084,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1085,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1086,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Split data

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tfidf_text, y, test_size=0.2, random_state=42)


Dalam Modelling kali ini, kita akan menggunakan model SVM dengan mengimport SVC dari library scikit-learn

In [18]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

Testing model, dan melihat akurasi

In [19]:
clf = SVC()
clf.fit(x_train, y_train)
pred = clf.predict(x_test)
accuracy = accuracy_score(y_test, pred)
print('accuracy score : ', accuracy)

accuracy score :  0.9678899082568807


Menyimpan Model tfidf dan model predict

In [20]:
import pickle
from pickle import dump
from pickle import load

tfidfvect = 'tfidf_vect.pkl'

# Menyimpan hasil normalisasi z_scaler ke dalam file pickle
with open(tfidfvect, 'wb') as file:
    pickle.dump(tfidf_fit, file)

In [21]:
model = 'svm.pkl'

# Menyimpan hasil normalisasi z_scaler ke dalam file pickle
with open(model, 'wb') as file:
    pickle.dump(clf, file)

Membuat fungsi klasifikasi sederhana

In [22]:
def prediksi(text):
  text = preprocessing_text(text)
  norm = tfidf_fit.transform([text])
  y_pred = clf.predict(norm)
  return y_pred
a = 'Sedari dulu golf bukanlah olahraga yang ecek-ecek, golf merupakan simbol dari gaya hidup mewah dan prestise. Karena melalui golf bisa mendapatkan pengalaman berolahraga santai di alam terbuka, serta kesempatan membangun jaringan sosial yang luas.Saat bermain golf, bukan hanya tubuh saja yang terasa lebih bugar, tapi juga mendapat ketenangan melalui pemandangan alam terbuka yang membuat tubuh dan pikiran jadi lebih rileks. Karena bermain golf perlu berjalan kaki sepanjang lapangan golf, membawa tas golf, dan melakukan pukulan yang memerlukan kekuatan dan keseimbangan. Beberapa jam bermain golf bisa membakar kalori dan memperkuat otot, membuatnya menjadi alternatif olahraga yang menyenangkan untuk menjaga kesehatan dan kebugaran.Fungsi lain dari olahraga golf selain untuk kesehatan adalah sebagai fasilitas jaringan koneksi dan hubungan sosial. Lapangan golf acap kali menjadi tempat pertemuan bisnis penting atau menjadi pusat komunitas berbagi minat. Itulah sebabnya golf juga dianggap sebagai olahraga yang ekslusif, investasi hiburan, bisnis, dan gaya hidup mewah. Di era sekarang dengan perkembangan teknologi dan inovasi yang sangat cepat juga turut berdampak ke dunia olahraga. Secara khusus pada cabang olahraga golf, peralatan masa kini telah dilengkapi dengan teknologi canggih untuk membantu meningkatkan performa permainanPeralatan golf masa kini banyak sekali memiliki kecanggihan, contohnya seperti perangkat lunak analitik untuk meningkatkan pukulan hingga penggunaan drone untuk pemetaan lapangan. Dunia golf terus mengikuti perkembangan zaman dengan menggabungkan teknologi untuk meningkatkan pengalaman bermain.Untuk mendapatkan peralatan golf terbaru dengan teknologi terkini, Bank Mega menawarkan pengalaman eksklusif dengan berbagai keuntungan dan diskon khusus untuk pemain golf sekaligus pemegang kartu kredit. Sekarang, Anda dapat menikmati kenyamanan golf mewah dengan diskon eksklusif dari Kartu Kredit Bank Mega. Tidak hanya itu, Anda dapat melakukan pengajuan aplikasi kartu kredit Bank Mega secara online untuk memperoleh Mega Travel Card, Mega Visa Platinum, Mega Visa Gold, atau Mega Metro Card.Selain untuk membeli peralatan baru, Bank Mega juga memberikan keuntungan eksklusif melalui kartu kredit untuk meningkatkan kenyamanan para penggemar golf. Fasilitas ini diberikan karena tidak semua lapangan golf menawarkan pengalaman mewah yang dapat membuat pengalaman bermain golf semakin istimewa. Anda dapat menikmati diskon eksklusif untuk berbagai fasilitas dan layanan di lapangan golf yang bekerja sama dengan Bank Mega.. Kartu kredit Bank Mega memberikan akses istimewa untuk meningkatkan pengalaman permainan golf Anda.Proses pengajuan kartu kredit Bank Mega yang mudah dan cepat, serta bisa diajukan secara online melalui website Bank Mega maupun melalui aplikasi M-Smile yang memungkinkan Anda untuk memilih jenis kartu kredit yang sesuai dengan kebutuhan dan gaya hidup Anda. Berikut ini 4 jenis kartu kredit Bank Mega yang dapat Anda pilih untuk mendapatkan fasilitas mewah dan kenyamanan saat berolahraga golf.Kartu Mega Travel memberikan Anda kebebasan untuk bepergian menikmati pemandangan sekaligus bermain golf di Bukit Pandawa Golf Bali atau Finna Golf & Country Club Pasuruan. Tersedia berbagai manfaat eksklusif, termasuk asuransi perjalanan, akses lounge bandara, dan promo diskon hingga 50% di restoran dan hotel.Selain untuk menjajal kemampuan di lapangan Pondok Indah Golf kartu kredit Mega Visa Platinum memberikan Anda keuntungan berlimpah, seperti reward poin yang dapat ditukarkan dengan berbagai hadiah, perlindungan asuransi, dan penawaran eksklusif di merchant-merchant pilihan.Dengan kartu Mega Visa Gold, Anda dapat menikmati diskon 10% saat bermain golf di New Kuta Golf Bali dan promo khusus di berbagai merchant favorit. Selain itu, kartu ini juga dilengkapi dengan berbagai fitur asuransi perlindungan.Kartu Mega Metro memberikan Anda kemudahan dalam bertransaksi sehari-hari dengan bonus poin dan diskon hingga 50% di berbagai merchant. Selain itu, Anda juga dapat menikmati akses ke promo eksklusif yang disediakan oleh Bank Mega.Melalui Mega Travel Card, Mega Visa Platinum, Mega Visa Gold, atau Mega Metro Card, Anda tidak hanya akan merasakan sensasi golf yang istimewa, tetapi juga menikmati berbagai keuntungan eksklusif dari Bank Mega. Jika Anda yang ingin menggabungkan kenyamanan golf mewah dengan keuntungan eksklusif kartu kredit, segera apply kartu kredit Bank Mega secara online. Manfaatkan kesempatan ini untuk membuat setiap putaran golf menjadi pengalaman tak terlupakan!'
prd = prediksi(a)
print(prd)


['sport']
